In [2]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
from dist_yolo.dist_yolo import DistYOLO
import cv2

In [2]:
dist_yolo = DistYOLO()
dist_yolo.load_model('./dist_yolo/trained_final (1).h5')
# image = Image.open(input_image_path)
# pred = dist_yolo.detect(image)
# print(pred)

In [10]:
input_image_path = r"./images/input7.png"
img = Image.open(input_image_path)
# result = dist_yolo.detect_image(img)
# result.show()
result = dist_yolo.detect(img)
dets = list(zip(result[0], result[1], result[2], result[3]))

1/1 [==============================] - 1s 734ms/step


In [11]:
# draw bounding box
output_image = Image.open(input_image_path)
draw = ImageDraw.Draw(output_image)
for p in dets:
    if p[1] != 1 and p[1] != 2 and p[1] != 3:
        continue
    
    draw.rectangle(((p[0][0], p[0][1]), (p[0][2], p[0][3])))
    draw.text(((p[0][0] + p[0][2]) / 2, (p[0][1] + p[0][3]) / 2), f'{p[3]}')

# output_image.save('output.jpg', 'JPEG')
output_image.show()

In [ ]:
x = cv2.imread()
Image.fromarray


Video frame count = 60
Number of CPU: 8


1/1 [==============================] - 1s 1s/step


In [11]:
# Draw bounding box to video
file_name = "video/video1.mp4"
output_file_name = "output/video1_output.mp4"

cap = cv2.VideoCapture(file_name)
with open('output/dist_yolo_output_video1.json') as f:
    dyo = json.load(f)['final_frames_detections']

fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
fps = int(cap.get(cv2.CAP_PROP_FPS))
width, height = (
            int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
)
out = cv2.VideoWriter()
out.open(output_file_name, fourcc, fps, (width, height), True)

proc_frames = 0
try:
    while proc_frames < len(dyo):
        ret, frame = cap.read()
        if not ret:
            break

        im = frame
        # Loop through list (if empty this will be skipped) and overlay green bboxes
        for i in dyo[proc_frames]:
            cv2.rectangle(im, (i[0], i[1]), (i[2], i[3]), (0, 255, 0), 3)
            cv2.putText(im, f'{int(i[6])}', ((i[0] + i[2])//2, (i[1]+i[3])//2), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        # write the frame
        out.write(im)

        proc_frames += 1
except Exception as error:
    # Release resources
    print(error)
    cap.release()
    out.release()

out.release()
cap.release()